# Search space overlap optimisation

* Find shapes within an irregular polygon search space, maximising overlap and minimising polygon area.

In [ ]:
from platypus import NSGAII, Problem, Real, nondominated
import pandas as pd
import numpy as np
from matplotlib import path
from shapely.geometry import Polygon
from descartes import PolygonPatch
import pylab as pl
from shapely import affinity
import matplotlib.pyplot as plt

In [ ]:
class Test(Problem):
    def __init__(self, lat_limits, lon_limits, map_polygon):
        super(Test, self).__init__(11, 2)

        self.lat_limits = lat_limits
        self.lon_limits = lon_limits

        lat_range = lat_limits[1] - lat_limits[0]
        lon_range = lon_limits[1] - lon_limits[0]

        # only predict locations within the site boundaries
        # allow predictions to be Real numbers (i.e. floats)
        self.types = [
            # polygon is rotated by rotation angle
            Real(0, 360),

            # center of site
            Real(self.lon_limits[0], self.lon_limits[1]),  # center lat
            Real(self.lat_limits[0], self.lat_limits[1]),  # center lon

            # corners are offset from center coordinates

            Real(-lon_range, 0),    # q1 lon
            Real(0, lat_range),     # q1 lat
            
            Real(0, lon_range),     # q2 lon
            Real(0, lat_range),     # q2 lat

            Real(0, lon_range),     # q3 lon
            Real(-lat_range, 0),    # q3 lat

            Real(-lon_range, 0),    # q4 lon
            Real(-lat_range, 0),    # q4 lat
        ]

        self.directions = [
            Problem.MINIMIZE,  # minimise area used
            Problem.MAXIMIZE,  # maximise intersection area
        ]

        self.map_polygon = map_polygon

    def evaluate(self, solution):
        """Evaluate variables of solution to determine fitness.

        Solution variables contain the following:
            rotation: rotation of the polygon about the center point.

            center_lat: latitude of the center point.
            center_lon: longitude of the center point.

            q1_lat: latitude of the first point of the quadrilateral.
            q1_lon: longitude of the first point of the quadrilateral.
            q2_lat: latitude of the second point of the quadrilateral.
            q2_lon: longitude of the second point of the quadrilateral.
            q3_lat: latitude of the third point of the quadrilateral.
            q3_lon: longitude of the third point of the quadrilateral.
            q4_lat: latitude of the fourth point of the quadrilateral.
            q4_lon: longitude of the fourth point of the quadrilateral.
        """

        x = solution.variables[:]

        # read solution into variables
        rotation = x[0]
        center = x[1:3]
        q1, q2, q3, q4 = x[3:5], x[5:7], x[7:9], x[9:11]
        
        # calculate corner coordinates from center and q offset
        q1_coord = [center[0] + q1[0], center[1] + q1[1]]
        q2_coord = [center[0] + q2[0], center[1] + q2[1]]
        q3_coord = [center[0] + q3[0], center[1] + q3[1]]
        q4_coord = [center[0] + q4[0], center[1] + q4[1]]

        # rotation corner coordinates by rotation angle
        points = np.array([q1_coord, q2_coord, q3_coord, q4_coord])

        shapely_polygon = Polygon(points).buffer(0)

        # rotate polygon by rotation angle about center coordinate
        shapely_polygon = affinity.rotate(shapely_polygon, rotation, origin=center)

        # calculate intersection area between map polygon and solution polygon
        intersection = self.map_polygon.intersection(shapely_polygon).area
        intersection = intersection / shapely_polygon.area * 100.0

        area = shapely_polygon.area * 100

        solution.objectives[:] = [
            area,
            intersection,
        ]


In [ ]:
# get size of search space
data = pd.read_csv("data/shape.csv")

data_lat_range = (min(data["Lat"]), max(data["Lat"]))
data_lon_range = (min(data["Lon"]), max(data["Lon"]))

print(f"Latitude range: {data_lat_range}, Longitude range: {data_lon_range}")


In [ ]:
# read map polygon coords and create polygon
polygon_df = pd.read_csv("data/alpha shape 17.5.csv")
polygon_np = polygon_df.to_numpy()

# swap columns (lon, lat) to (lat, lon)
polygon_np = polygon_np[:, [1, 0]]

# convert to shapely polygon
map_polygon = Polygon(polygon_np)


In [ ]:
# load the problem
problem = Test(
    lat_limits=data_lat_range,
    lon_limits=data_lon_range,
    map_polygon=map_polygon,
)


In [ ]:
# run the optimisation
algorithm = NSGAII(problem)
algorithm.run(1000)


In [ ]:
# find non-dominated solutions
nondominated_solutions = nondominated(algorithm.result)
Y = np.array([s.objectives for s in nondominated_solutions])


In [ ]:
# convert results to pandas dataframe
results = pd.DataFrame(
    columns=[
        "rotation",
        "center_lon",
        "center_lat",
        "q1_lon",
        "q1_lat",
        "q2_lon",
        "q2_lat",
        "q3_lon",
        "q3_lat",
        "q4_lon",
        "q4_lat",
        "Area",
        "Intersection",
    ]
)

# read nondominated solutions into dataframe
for i, solution in enumerate(nondominated_solutions):
    results.loc[i] = [*solution.variables[:], *solution.objectives]


Plot pareto front of solutions

In [ ]:
fig = plt.figure(figsize = (10, 7))
ax = plt.axes(projection ="3d")
ax.scatter3D(Y[:, 0], Y[:, 1], Y[:, 2])
ax.set_xlabel('Area')
ax.set_ylabel('Intersection')
plt.show()


Create folder of top 15 results with >80% intersection

In [ ]:
fig = pl.figure(figsize=(10, 10))
ax = fig.add_subplot(111)

# load depth scatter into plot and reuse
map = ax.scatter(data["Lon"], data["Lat"], c=data["Depth"], cmap="viridis")

ax.set_xlabel("Lat")
ax.set_ylabel("Lon")

map_patch = PolygonPatch(map_polygon, fc="#999999", ec="#000000", fill=False)
ax.add_patch(map_patch)

# filter to get top 15 solutions with intersection > 80%
results_filtered = results[results["Intersection"] > 80]
results_filtered = results_filtered.sort_values(by="Intersection", ascending=False)
results_filtered = results_filtered.head(15)
results_filtered = results_filtered.reset_index(drop=True)

for idx, x in results_filtered.iterrows():
    # find the best solution
    x = results.loc[idx]
    x = x.reset_index(drop=True)

    x = x.to_numpy()

    # set title
    ax.set_title(f"Depth: {x[11]:.2f}m, Wind Potential: {x[12]:.2f}m/s, Area: {x[13]:.2f}%, Intersection: {x[14]:.2f}%")

    # read solution into variables
    rotation = x[0]
    center = x[1:3]
    q1, q2, q3, q4 = x[3:5], x[5:7], x[7:9], x[9:11]
    
    # calculate corner coordinates from center and q offset
    # REVERSE XY from LAT LON to LON LAT
    q1_coord = [center[0] + q1[0], center[1] + q1[1]]
    q2_coord = [center[0] + q2[0], center[1] + q2[1]]
    q3_coord = [center[0] + q3[0], center[1] + q3[1]]
    q4_coord = [center[0] + q4[0], center[1] + q4[1]]

    # rotation corner coordinates by rotation angle
    points = np.array([q1_coord, q2_coord, q3_coord, q4_coord])

    polygon = Polygon(points)
    # rotate polygon by rotation angle about center coordinate
    polygon = affinity.rotate(polygon, rotation, origin=center)

    center_point = ax.scatter(center[0], center[1], c="r", marker="*")

    patch = PolygonPatch(polygon, fc="#999999", ec="#000000", fill=False)
    ax.add_patch(patch)

    # save figure
    fig.savefig(f"results/{idx}.png")

    # reset patches for next iteration
    patch.remove()
    center_point.remove()
